In [1]:
import pandas as pd
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"

MONTHS = [
    "Janeiro",
    "Fevereiro",
    "Março",
    "Abril",
    "Maio",
    "Junho",
    "Julho",
    "Agosto",
    "Setembro",
    "Outubro",
    "Novembro",
    "Dezembro"
]

STATES = {
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AP': 'Amapá',
    'AM': 'Amazonas',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MT': 'Mato Grosso',
    'MS': 'Mato Grosso do Sul',
    'MG': 'Minas Gerais',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PR': 'Paraná',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'SC': 'Santa Catarina',
    'SP': 'São Paulo',
    'SE': 'Sergipe',
    'TO': 'Tocantins'
}

# caminho de alguns arquivos
IMPORT_2020_FILE_PATH = "data/IMP_2020_V2.csv"
EXPORT_2020_FILE_PATH = "data/EXP_2020.csv"
PRODUCTS_MAPPING_FILE_PATH = "data/d_sh2.xlsx"

# nomes personalizados de atributos
IMPORT_COLUMNS = [
    "ANO", "MES", "COD_NCM", "COD_UNIDADE",
    "COD_PAIS", "SG_UF", "COD_VIA",
    "COD_URF", "VL_QUANTIDADE", "VL_PESO_KG",
    "VL_FOB", "VL_FRETE","VL_SEGURO"
]

EXPORT_COLUMNS = [
    "ANO", "MES", "COD_NCM",
    "COD_UNIDADE", "COD_PAIS",
    "SG_UF", "COD_VIA", "COD_URF",
    "QT_ESTAT", "VL_PESO_KG", "VL_FOB"
]
# opções de indicadores
INDICATOR_OPTIONS = ["QT_ESTAT", "VL_PESO_KG", "VL_FOB"]

### Dados financeiros

In [3]:
# dados de importação
import_dataset = pd.read_csv(IMPORT_2020_FILE_PATH,  delimiter=";")
import_dataset.columns = IMPORT_COLUMNS
import_dataset['MOVIMENTACAO'] = "importação"

# dados de exportação
export_dataset = pd.read_csv(EXPORT_2020_FILE_PATH,  delimiter=";")
export_dataset.columns = EXPORT_COLUMNS
export_dataset['MOVIMENTACAO'] = "exportação"

# Concatenando os dados
frames = [export_dataset, import_dataset]
dataset = pd.concat(frames)

# salvando um pouco de memória
del export_dataset
del import_dataset
del frames

In [15]:
dataset.head()

,ANO,MES,COD_NCM,COD_UNIDADE,COD_PAIS,SG_UF,COD_VIA,COD_URF,QT_ESTAT,VL_PESO_KG,VL_FOB,MOVIMENTACAO,VL_QUANTIDADE,VL_FRETE,VL_SEGURO
0,2020,1,39211900,10,244,SP,4,817600,1.0,1,2,exportação,NaN,NaN,NaN
1,2020,3,48183000,10,573,SC,1,917800,1.0,1,69,exportação,NaN,NaN,NaN
2,2020,1,85364100,11,756,SP,1,817800,341.0,12,1668,exportação,NaN,NaN,NaN
3,2020,7,69109000,11,586,SP,7,147800,13.0,8,686,exportação,NaN,NaN,NaN
4,2020,6,87089990,10,97,SP,7,147600,5960.0,5960,49740,exportação,NaN,NaN,NaN


### Mapeamento de categoria de produtos

In [ ]:
products_mapping_df = pd.read_excel(PRODUCTS_MAPPING_FILE_PATH)
products_mapping_df.head()

### Processamento para o primeiro plot (Gráfico de barras para os indicadores de importação/exportação)

In [ ]:
filters = (dataset["ANO"] == 2020) & (dataset["MOVIMENTACAO"] == "exportação") & (dataset["COD_NCM"] == 39211900)
filtered = dataset[filters]
data =filtered.groupby(by='MES')[metric_options].sum()
data.reset_index(level=0, inplace=True) # remove MES como index externo

### Processamento para o segundo plot (Gráfico de pizza para o uso percentual da via)

In [ ]:
a = pd.DataFrame(
    {
     "code": filtered["COD_VIA"].value_counts().index,
     "count": filtered["COD_VIA"].value_counts().values
    }
)
total = a["count"].sum()
a['as_percentage'] = a["count"].apply(lambda x: x/total)
a

### Processamento para a tabela de contribuição por estado

In [18]:
# agrupa por estado e soma os totais
total_by_state = dataset.groupby(["SG_UF"])[INDICATOR_OPTIONS].sum()
total_by_state.reset_index(level=0, inplace=True) # remove SG_UF como index externo

# calcula os percentuais por estado
for indicator in INDICATOR_OPTIONS:
    # nome da nova coluna com a contribuição percenutal
    new_col_name = f"{indicator}_CONTRIB_%"
    
    # contribuição absoluta por estado
    total = total_by_state[indicator].sum()
    total_by_state[new_col_name] = total_by_state[indicator].apply(lambda x: round(x/total * 100, 2))
    
# Substitui a sigla pelo nome do estado
total_by_state['SG_UF'] = total_by_state["SG_UF"].apply(lambda x: STATES.get(x,"Exportação"))
total_by_state.head(10)

,SG_UF,QT_ESTAT,VL_PESO_KG,VL_FOB,VL_QUANTIDADE,VL_FRETE,VL_SEGURO,QT_ESTAT_CONTRIB_%,VL_PESO_KG_CONTRIB_%,VL_FOB_CONTRIB_%,VL_QUANTIDADE_CONTRIB_%,VL_FRETE_CONTRIB_%,VL_SEGURO_CONTRIB_%
0,Acre,2.621058e+07,51409279,36808531,1.711841e+06,219117.0,1008.0,0.01,0.01,0.01,0.00,0.00,0.00
1,Alagoas,7.053886e+07,2302732712,1083758019,1.286341e+09,61038327.0,496831.0,0.01,0.27,0.29,0.55,0.82,0.43
2,Amazonas,7.541154e+08,2619743134,10504948567,5.444528e+10,520806621.0,4541910.0,0.15,0.31,2.85,23.37,7.01,3.90
3,Amapá,6.692512e+08,1138125207,493862955,8.119802e+07,14502816.0,120810.0,0.13,0.14,0.13,0.03,0.20,0.10
4,Bahia,4.956223e+09,21635148295,12809398883,5.035462e+09,297584793.0,3111646.0,0.98,2.57,3.48,2.16,4.00,2.68
5,Ceará,3.496833e+09,10056499672,4266966681,6.301701e+09,186933638.0,2001505.0,0.69,1.20,1.16,2.70,2.51,1.72
6,Distrito Federal,7.983821e+07,411049325,1506810299,2.161441e+08,58311286.0,2080019.0,0.02,0.05,0.41,0.09,0.78,1.79
7,Espírito Santo,1.777202e+10,27182251394,10018298001,6.154733e+09,219042947.0,5264611.0,3.53,3.23,2.72,2.64,2.95,4.53
8,Exportação,0.000000e+00,3644,169061,3.400000e+01,8935.0,1035.0,0.00,0.00,0.00,0.00,0.00,0.00
9,Goiás,5.351381e+09,19741758957,11453098514,3.698683e+09,147456589.0,3452674.0,1.06,2.35,3.11,1.59,1.98,2.97
